In [ ]:
# Projeto: Previsão de Demanda Mensal por Produto com Lags
# Autor: Alexandre Teixeira
# Descrição: Este notebook realiza a previsão da demanda mensal de produtos por fornecedor,
#            utilizando atributos de atraso (lags) e modelos de regressão com validação temporal.
#            A previsão é feita para o próximo mês com base na demanda dos dois meses anteriores.
# Dataset: orders_2.csv, orders_products_2.csv, prices_2.csv
# Data da última atualização: Julho/2025

import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_absolute_error

# === 1. Carregamento dos dados ===
orders = pd.read_csv('/content/drive/MyDrive/novas_tabelas_2/orders_2.csv', parse_dates=['data_pedido'])
order_products = pd.read_csv('/content/drive/MyDrive/novas_tabelas_2/orders_products_2.csv', parse_dates=['data_preco'])
prices = pd.read_csv('/content/drive/MyDrive/novas_tabelas_2/prices_2.csv')

# === 2. Merge dos dados ===
merged = order_products.merge(
    orders[['id', 'data_pedido']],
    left_on='id_pedido',
    right_on='id',
    how='inner'
)

merged = merged.merge(
    prices[['id_preco', 'valor']],
    on='id_preco',
    how='left'
)

# === 3. Agregação mensal ===
merged['order_month'] = merged['data_pedido'].dt.to_period('M').astype(str)
demand_prod_month = (
    merged.groupby(['id_produto_fornecedor', 'order_month'])['quantidade']
    .sum()
    .reset_index(name='monthly_quantity')
)

# === 4. Criação de features de lag ===
demand_prod_month = demand_prod_month.sort_values(['id_produto_fornecedor', 'order_month'])
demand_prod_month['prev_month_quantity'] = demand_prod_month.groupby('id_produto_fornecedor')['monthly_quantity'].shift(1)
demand_prod_month['prev2_month_quantity'] = demand_prod_month.groupby('id_produto_fornecedor')['monthly_quantity'].shift(2)
demand_prod_month[['prev_month_quantity', 'prev2_month_quantity']] = demand_prod_month[['prev_month_quantity', 'prev2_month_quantity']].fillna(0)

# === 5. Preparação para modelagem ===
X = demand_prod_month[['prev_month_quantity', 'prev2_month_quantity']]
y = demand_prod_month['monthly_quantity']

# === 6. Validação com Time Series Split ===
tscv = TimeSeriesSplit(n_splits=3)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# === 7. Pipelines de modelos ===
pipeline_ridge = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge(alpha=1.0, random_state=42))
])

pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipeline_hgb = Pipeline([
    ('scaler', StandardScaler()),
    ('hgb', HistGradientBoostingRegressor(random_state=42))
])

# === 8. Avaliação dos modelos ===
models = {
    'Ridge': pipeline_ridge,
    'RandomForest': pipeline_rf,
    'HistGradientBoosting': pipeline_hgb
}

results = {}
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=tscv, scoring=mae_scorer)
    results[name] = (-scores.mean(), scores.std())

print("\nDesempenho (MAE) dos modelos:")
for name, (mean_mae, std_mae) in results.items():
    print(f"{name}: MAE = {mean_mae:.2f} ± {std_mae:.2f}")

# === 9. Treinamento final ===
demand_prod_month['order_month'] = pd.to_datetime(demand_prod_month['order_month'])
df_processed = demand_prod_month.set_index('order_month')
X = df_processed[['prev_month_quantity', 'prev2_month_quantity']]
y = df_processed['monthly_quantity']
pipeline_hgb.fit(X, y)

# === 10. Preparar entrada para previsão ===
last_month = df_processed.index.max()
next_month = last_month + pd.DateOffset(months=1)
second_last_month = last_month - pd.DateOffset(months=1)

processed_reset = df_processed.reset_index()
current = processed_reset[processed_reset['order_month'] == last_month][['id_produto_fornecedor', 'monthly_quantity']]
current = current.rename(columns={'monthly_quantity': 'prev_month_quantity'})

previous = processed_reset[processed_reset['order_month'] == second_last_month][['id_produto_fornecedor', 'monthly_quantity']]
previous = previous.rename(columns={'monthly_quantity': 'prev2_month_quantity'})

prediction_input = current.merge(previous, on='id_produto_fornecedor', how='left')
prediction_input = prediction_input.fillna(0)
X_predict = prediction_input[['prev_month_quantity', 'prev2_month_quantity']]
predicted = pipeline_hgb.predict(X_predict)

predictions_df = pd.DataFrame({
    'id_produto_fornecedor': prediction_input['id_produto_fornecedor'],
    'mes_previsao': next_month.strftime('%Y-%m'),
    'demanda_prevista': predicted.round(0).astype(int)
})

print(f"\nPrevisões para {next_month.strftime('%B de %Y')}:\n")
print(predictions_df)